In [2]:
#! pip install datasets transformers
! pip install transformers

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 35.8 MB/s 
     |████████████████████████████████| 596 kB 47.9 MB/s 
     |████████████████████████████████| 67 kB 3.7 MB/s 
     |████████████████████████████████| 6.5 MB 39.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import transformers

print(transformers.__version__)

4.17.0


## Importing necessary modules

In [4]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

## Preparing our dataset

In [11]:
!wget -L 'https://raw.githubusercontent.com/ankitverma5859/IIT_KGP/main/SprintSemester2022/CN_Project/qutus_dataset.json'

--2022-04-02 17:39:14--  https://raw.githubusercontent.com/ankitverma5859/IIT_KGP/main/SprintSemester2022/CN_Project/qutus_dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256488 (250K) [text/plain]
Saving to: ‘qutus_dataset.json.1’

qutus_dataset.json. 100%[===================>] 250.48K  --.-KB/s    in 0.03s   

2022-04-02 17:39:15 (7.84 MB/s) - ‘qutus_dataset.json.1’ saved [256488/256488]



In [22]:
!wget -L 'https://raw.githubusercontent.com/agnik2019/cn_project/main/qutus_dataset.json'

--2022-04-02 17:51:39--  https://raw.githubusercontent.com/agnik2019/cn_project/main/qutus_dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169456 (165K) [text/plain]
Saving to: ‘qutus_dataset.json’

qutus_dataset.json  100%[===================>] 165.48K  --.-KB/s    in 0.02s   

2022-04-02 17:51:39 (6.89 MB/s) - ‘qutus_dataset.json’ saved [169456/169456]



In [17]:
import pandas as pd
import json

In [23]:
df = pd.read_json('/content/qutus_dataset.json')

In [24]:
df.head()

,answers,context,id,question,title
0,"{'answer_start': [265], 'text': ['when the Uni...",What a different speech this could have been. ...,1,The presentation opened with a series of banal...,<OPTION>
1,"{'answer_start': [190], 'text': ['When the mar...",The presentation opened with a series of banal...,2,"“When the Union was turned back at Bull Run, a...",<OPTION>
2,"{'answer_start': [111], 'text': ['Again, we ar...","While more dark days will come, things are get...",3,"While more dark days will come, things are get...",<OPTION>
3,"{'answer_start': [295], 'text': ['One in ten A...",The 2 blue-uniformed personnel in the First la...,4,Is Obama going have them stand later as exampl...,<OPTION>
4,"{'answer_start': [], 'text': ['One in 10 Ameri...","In his first State of the Union address, Presi...",5,"He gave voice to our hopes and values, but he ...",<OPTION>


In [41]:
len(df)

183

In [44]:
answer_list = []
for i in range(len(df)):
  answer_list.append(df['answers'].to_dict()[i]['text'])

In [45]:
answer_list

[['when the Union was turned back at Bull Run, and the Allies first landed at Omaha Beach, victory was very much in doubt.'],
 ['When the market crashed on Black Tuesday, and civil rights marchers were beaten on Bloody Sunday, the future was anything but certain.'],
 ["Again, we are tested. And again, we must answer history's call."],
 ['One in ten Americans still cannot find work.'],
 ['One in 10 Americans still cannot find work. Many businesses have shuttered. Home values have declined. Small towns and rural communities have been hit especially hard. for those who already known poverty, life has become that much harder.'],
 ['Change has not come fast enough'],
 ['unable or unwilling to solve any of our problems.'],
 ['So we face big and difficult challenges.'],
 ['And what the American people hope- what they deserve- is for all of us, Democrats and Republicans, to work through our differences; to overcome the numbing weight of our politics.'],
 ['what the American people hope-what th

In [47]:
df['answer_list'] = answer_list

In [49]:
df_new = df[["answer_list","context","question"]]

In [50]:
df_new.head()

,answer_list,context,question
0,"[when the Union was turned back at Bull Run, a...",What a different speech this could have been. ...,The presentation opened with a series of banal...
1,"[When the market crashed on Black Tuesday, and...",The presentation opened with a series of banal...,"“When the Union was turned back at Bull Run, a..."
2,"[Again, we are tested. And again, we must answ...","While more dark days will come, things are get...","While more dark days will come, things are get..."
3,[One in ten Americans still cannot find work.],The 2 blue-uniformed personnel in the First la...,Is Obama going have them stand later as exampl...
4,[One in 10 Americans still cannot find work. M...,"In his first State of the Union address, Presi...","He gave voice to our hopes and values, but he ..."


### Building the chatbots

In [51]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [54]:
random_num = np.random.randint(0,len(df))
question = df["question"][random_num]
text = df["context"][random_num]

In [55]:
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 167 tokens.


In [56]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
america    2,637
is         2,003
a          1,037
critical   4,187
actor      3,364
and        1,998
leader     3,003
on         2,006
the        1,996
world      2,088
stage      2,754
and        1,998
that       2,008
we         2,057
shouldn    5,807
'          1,005
t          1,056
be         2,022
embarrassed  10,339
about      2,055
that       2,008
.          1,012
[SEP]        102
still      2,145
,          1,010
the        1,996
leaders    4,177
,          1,010
many       2,116
wary      15,705
of         1,997
pi        14,255
##ling     2,989
up         2,039
debt       7,016
,          1,010
did        2,106
not        2,025
sign       3,696
off        2,125
on         2,006
large      2,312
new        2,047
stimulus  19,220
packages  14,555
for        2,005
their      2,037
own        2,219
countries   3,032
.          1,012
obama      8,112
'          1,005
s          1,055
administration   3,447
had        2,018
initially   3,322
pushed     3,724
for

In [57]:
#first occurence of [SEP] token
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)
#number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)
#number of tokens in segment B (text)
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)
#creating the segment ids
segment_ids = [0]*num_seg_a + [1]*num_seg_b
#making sure that every input token has a segment id
assert len(segment_ids) == len(input_ids)

SEP token index:  23
Number of tokens in segment A:  24
Number of tokens in segment B:  143


In [58]:
#token input_ids to represent the input and token segment_ids to differentiate our segments - question and text
output = model(torch.tensor([input_ids]),  token_type_ids=torch.tensor([segment_ids]))

In [59]:
#tokens with highest start and end scores
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")
    
print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))


Question:
America is a critical actor and leader on the world stage and that we shouldn't be embarrassed about that.

Answer:
America is a critical actor and leader on the world stage.


In [60]:
answer = tokens[answer_start]
for i in range(answer_start+1, answer_end+1):
    if tokens[i][0:2] == "##":
        answer += tokens[i][2:]
    else:
        answer += " " + tokens[i]

In [61]:
def question_answer(question, text):
    
    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1
    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    print("\nPredicted answer:\n{}".format(answer.capitalize()))

In [62]:

text = input("Please enter your text: \n")
question = input("\nPlease enter your question: \n")
while True:
    question_answer(question, text)
    
    flag = True
    flag_N = False
    
    while flag:
        response = input("\nDo you want to ask another question based on this text (Y/N)? ")
        if response[0] == "Y":
            question = input("\nPlease enter your question: \n")
            flag = False
        elif response[0] == "N":
            print("\nBye!")
            flag = False
            flag_N = True
            
    if flag_N == True:
        break

Please enter your text: 
you all should work hard

Please enter your question: 
how to overcome

Predicted answer:
Work hard

Do you want to ask another question based on this text (Y/N)? N

Bye!


### Reference:
https://github.com/chetnakhanna16/CoQA_QuesAns_BERT/blob/main/CoQA_BERT_QuestionAnswering.ipynb